# Échauffement de Huffman

1. Donner la table des fréquences puis l'arbre de Huffman du texte suivant: `ceci est un exemple de code de Huffman, le construire c'est le maitriser`
2. Utiliser l'arbre de Huffman construit pour coder le texte (ou au moins ses première lettres) et comparer la taille du texte compressé à la taille du texte original en ASCII.

# Compression LZ77


L'algorithme marche de la façon suivante:

* On a une chaîne de caractères en entrée et on commence en se plaçant sur le premier caractère

* Sur un caractère donné on cherche le plus grand sous-mot qui commence par cette lettre et
  qui apparaît aussi avant. Par exemple si on est en troisième position dans an*a*nas alors le plus long sous-mot 
  est *ana*.
 
* On code ce sous-mot en faisant référence à sa précédente occurrence. Dans l'exemple précédent, 
  le sous-mot fait trois lettres et commence deux lettres avant, on le code par (2,3).

* Si le mot est de taille 0, ou 1, on écrit juste la lettre.


Généralement on borne la fenêtre de recherche dans laquelle on cherche un sous-mot identique afin de limiter
le temps de calcul quand on compresse de grands textes.

1. Donner le résultat de la compression de `aaaaaaaaaaaaaaaaa`.
2. Donner le résultat de la compression de `Chabadi chabada chabababa`.
3. Décompresser le message suivant: `Si, six(4, 2)cies(6, 5)nt(17, 5)cyprès, alor(25, 3)(18, 4)(27, 8)(44, 10)ro(46, 8)(55, 14)`.

# Implémentation

On va compléter le code suivant, pour construire un petit compresseur LZ77.



In [47]:
import tkinter as tk

taille_fenetre = 100 #Taille de la fenêtre de recherche

def match_size(mot,i,j):
    k = 0
    while i+k<len(mot) and mot[j+k] == mot[i+k]:
        k += 1 #renvoie la valeur du plus grand sous-mot commun,
                    #dans le mot aux positions position i et j avec j < i
    return k

def max_match(mot,i): #renvoie le couple (position,taille) du plus grand match 
                      #trouvé dans mot à partir de la position i
    j = max(0,i-taille_fenetre)  #première position où chercher un match
    max_match = (0,0)
    while j < i: #on cherche un match dans la fenetre de recherche
        size = match_size(mot,i,j)
        if size > max_match[1]:
            max_match = (j,size)
        j +=1
    return max_match


def compresse():
    texte_a_compresser = entree.get()
    texte_compresse = [] #cette liste doit etre étendue pour contenir le texte compressé
    #construction du code LZ77
    i = 0
    while i < len(texte_a_compresser): #pour chaque lettre du texte
        match = max_match(texte_a_compresser,i)
        if match[1] > 2:
            texte_compresse.append((i-match[0],match[1]))
            i += match[1]
        else:
            texte_compresse.append(texte_a_compresser[i])
            i+=1
    affichage_compression.config(text = str(texte_compresse)) #affichage du texte compressé 
    resultat.config(text ="Taille initiale de "+str(len(texte_a_compresser))+"\n"+ "Taille compressée de " + str(taille(texte_compresse))+"\nRatio : "+str(taille(texte_compresse)/len(texte_a_compresser)))


def taille(liste_LZ):# calcule la taille de la liste. Un caractère compte 1 et 
                                   # une paire d'entier compte 2
    taille = 0
    for elem in liste_LZ:
        if isinstance(elem,str):
            taille += 1
        else:
            taille += 2
    return taille


racine = tk.Tk()
racine.title("Compression de texte")


entree = tk.Entry(racine, width = 100,font = ("helvetica", "20"))
entree.grid(row = 0, column = 0)



affichage_compression = tk.Message(racine, font = ("helvetica", "20"), width = 1000)
affichage_compression.grid(row = 1, column = 0, columnspan = 2)

bouton_compresser = tk.Button(racine, text = "Compresser", command = compresse, font = ("helvetica", "30"))
bouton_compresser.grid(row = 0, column = 1)

resultat = tk.Label(racine, font = ("helvetica", "20"))
resultat.grid(row = 2, column = 0, columnspan = 2)


racine.mainloop()

1. Compléter la fonction `match_size(mot,i,j)` qui trouve le plus grand sous-mot commun aux positions `i` et `j`
   de la chaîne de caractère `mot`. On appelle un tel sous mot un *match*.
   
2. Compléter la fonction `max_match(mot,i)` qui trouve le plus grand match pour la position `i` de la chaîne de caractère `mot`. La fonction renvoie une paire d'entiers qui contient la position par rapport à `i` de ce match
   et sa taille.
3. Compléter la fonction `compresse` qui crée une liste de caratères et de paires d'entiers correspondant au code LZ77 du texte entrée, se trouvant dans la variable `texte_a_compresser`.
4. Compléter la fonction `taille` qui calcule la taille du texte compressé, en comptant 1 pour les lettres et 2 pour les paires d'entiers. Pour distinguer un caractère d'une paire d'entiers dans la liste, on pourra utiliser
   `isinstance(elem,str)` qui est vrai si et seulement si elem est une chaîne de caractère (les caractères en python sont des chaînes de caractère de taille 1).
5. Ajouter un label ou plusieurs labels pour afficher les informations suivantes: la taille de la fenêtre de recherche, la taille du texte original et le rapport de compression (taille/taille_compressée).
6. Permettre à l'utilisateur de changer la taille de la fenêtre de recherche. Que se passe-t-il quand on change sa valeur.
7. Que se passe-t-il quand on interdit les matchs de taille 2 ? Et les matchs de taille 3 ?
8. Est-ce que l'algorithme proposé trouve un code LZ optimal en terme de taille.
9. Proposer un codage binaire d'un code LZ77 et écrire le code qui affiche un tel code binaire.
   On supposera l'entrée donnée en ASCI (sur 7 bits). On pourra coder les entiers ainsi que les caractères sur 8 bits. Attention, il faut différencier les caractères et les paires d'entiers.


In [48]:

def decompresse(texte_compresse): #texte_compresse est une liste de caractère et
                                  # de paires d'entiers (ce qui est produit par compresse)
	res = [] # liste des caractères obtenus par decompression
	pos = 0  # position du caractère courant dans res
	for i in range(len(texte_compresse)):
		if isinstance(texte_compresse[i],str):
			# un caractère est lu, à compléter
			res.append(texte_compresse[i])
			pos += 1
		else:
            # une paire d'entiers est lue, à compléter
			decalage, taille = texte_compresse[i]
			res.extend(res[pos-decalage:pos-decalage+taille])
			pos += taille
	return "".join(res)

print(decompresse(["a","b","c",(3,6)]))

abcabc


Compléter le code précédent pour décoder la liste passé en argument.
Vous décoderez le code créé par la fonction `compresse` à l'aide de la fonction `decompresse` et vous afficherez le résultat dans un nouveau label.